<a href="https://colab.research.google.com/github/RoshanKumar854/ML-Projects/blob/main/Kaggle_Feedback_Prize_Predicting_Effective_Arguments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [250]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim
5,2e214524dbe3,D72CB1C11673,When making a decision there is a chance that ...,Evidence
6,84812fc2ab9f,D72CB1C11673,Everyone is different and may have more experi...,Evidence
7,c668ff840720,D72CB1C11673,Seeking others opinion can be very helpful and...,Claim
8,739a6d00f44a,D72CB1C11673,Taking other peoples advice and doing what the...,Evidence
9,bcfae2c9a244,D72CB1C11673,You can learn from others experiences by seeki...,Concluding Statement


In [251]:
train_data["discourse_text"] = train_data['discourse_text'].astype('str').str.replace(r".", r"", regex=False)
train_data["discourse_text"] = train_data['discourse_text'].astype('str').str.replace(r",", r"", regex=False)
test_data["discourse_text"] = test_data['discourse_text'].astype('str').str.replace(r".", r"", regex=False)
train_data["discourse_text"] = train_data['discourse_text'].astype('str').str.replace(r"'", r"", regex=False)

In [252]:
train_data["discourse_text"]

0        Hi im Isaac im going to be writing about how t...
1        On my perspective I think that the face is a n...
2        I think that the face is a natural landform be...
3        If life was on Mars we would know by now The r...
4        People thought that the face was formed by ali...
                               ...                        
36760    For many people they dont like only asking one...
36761    also people have different views and opinions ...
36762    Advice is something that can impact a persons ...
36763    someone can use everything that many people sa...
36764    In conclusion asking for an opinion can be ben...
Name: discourse_text, Length: 36765, dtype: object

In [253]:
train_data

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,Hi im Isaac im going to be writing about how t...,Lead,Adequate
1,9704a709b505,007ACE74B050,On my perspective I think that the face is a n...,Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,If life was on Mars we would know by now The r...,Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they dont like only asking one...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [254]:
train_data = train_data.drop(["discourse_id", "essay_id"], axis=1)
train_data

,discourse_text,discourse_type,discourse_effectiveness
0,Hi im Isaac im going to be writing about how t...,Lead,Adequate
1,On my perspective I think that the face is a n...,Position,Adequate
2,I think that the face is a natural landform be...,Claim,Adequate
3,If life was on Mars we would know by now The r...,Evidence,Adequate
4,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...
36760,For many people they dont like only asking one...,Claim,Adequate
36761,also people have different views and opinions ...,Claim,Adequate
36762,Advice is something that can impact a persons ...,Position,Adequate
36763,someone can use everything that many people sa...,Evidence,Ineffective


In [255]:
MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_data['discourse_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31515 unique tokens.


In [256]:
X = tokenizer.texts_to_sequences(train_data['discourse_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (36765, 250)


In [257]:
Y = pd.get_dummies(train_data['discourse_effectiveness']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (36765, 3)


In [258]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
517/517 [==============================] - 308s 590ms/step - loss: 0.8657 - accuracy: 0.6209 - val_loss: 0.8746 - val_accuracy: 0.6078
Epoch 2/5
517/517 [==============================] - 297s 575ms/step - loss: 0.7714 - accuracy: 0.6570 - val_loss: 0.8818 - val_accuracy: 0.6095
Epoch 3/5
517/517 [==============================] - 300s 580ms/step - loss: 0.7296 - accuracy: 0.6785 - val_loss: 0.8700 - val_accuracy: 0.6133
Epoch 4/5
517/517 [==============================] - 298s 576ms/step - loss: 0.6984 - accuracy: 0.6932 - val_loss: 0.9302 - val_accuracy: 0.5942
Epoch 5/5
517/517 [==============================] - 298s 576ms/step - loss: 0.6742 - accuracy: 0.7074 - val_loss: 1.0376 - val_accuracy: 0.5836


In [262]:
test_data

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult P...,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial,Claim
5,2e214524dbe3,D72CB1C11673,When making a decision there is a chance that ...,Evidence
6,84812fc2ab9f,D72CB1C11673,Everyone is different and may have more experi...,Evidence
7,c668ff840720,D72CB1C11673,Seeking others opinion can be very helpful and...,Claim
8,739a6d00f44a,D72CB1C11673,Taking other peoples advice and doing what the...,Evidence
9,bcfae2c9a244,D72CB1C11673,You can learn from others experiences by seeki...,Concluding Statement


In [263]:
MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(test_data['discourse_text'].values)
word_index = tokenizer.word_index

X_test = tokenizer.texts_to_sequences(test_data['discourse_text'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)



In [302]:
predictions = model.predict(X_test)

In [303]:
predictions = pd.DataFrame(predictions)
predictions.columns = ['Adequate', 'Effective', 'Ineffective']
predictions.insert(0,"discourse_id", test_data["discourse_id"], True)
new_cols = ["discourse_id","Ineffective","Adequate","Effective"]
predictions=predictions[new_cols]

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.285710,0.636916,0.077375
1,5a88900e7dc1,0.289812,0.672650,0.037538
2,9790d835736b,0.182405,0.760435,0.057159
3,75ce6d68b67b,0.327021,0.630532,0.042446
4,93578d946723,0.192516,0.754549,0.052934
5,2e214524dbe3,0.338871,0.629248,0.031881
6,84812fc2ab9f,0.154212,0.795380,0.050408
7,c668ff840720,0.298992,0.686695,0.014313
8,739a6d00f44a,0.072053,0.561071,0.366876
9,bcfae2c9a244,0.351193,0.629617,0.019190


In [304]:
predictions.to_csv("predictions.csv")